# Chapter2 言語資源と言語モデル

## 2.1
> 上に述べたようにシソーラスやオントロジーには様々なものがある. インターネットを利用してそれらのシソーラスを利用することができるので, それぞれのシソーラスの違いを調べてみよう. どのような違いが見られ, またその違いはなぜ生じるのか考えてみよう.

- [WordNet](https://bond-lab.github.io/wnja/jpn/index.html)
- [NINJAL](https://clrd.ninjal.ac.jp/goihyo.html)

In [16]:
import os
import sqlite3
import pandas as pd

In [124]:
if not os.path.isfile("wnjpn.db"):
    os.system("wget https://github.com/bond-lab/wnja/releases/download/v1.1/wnjpn.db.gz")
    os.system("gzip -d wnjpn.db.gz")
if not os.path.isfile("bunruidb.txt"):
    os.system("wget https://github.com/masayu-a/WLSP/raw/master/bunruidb.txt")

In [125]:
class WordNet: 
    def __init__(self) -> None:
        self.connect()
    
    def connect(self) -> None:
        self.con = sqlite3.connect("wnjpn.db")
        self.cur = self.con.cursor()

    def close(self) -> None:
        self.cur.close()
        self.con.close()

    def read_sql(self, SQL_statement: str) -> pd.DataFrame:
        return pd.read_sql(SQL_statement, self.con) 

    def show_tables(self, SQL_statement: str) -> list:
        res = self.con.execute("SELECT table FROM sqlite_master WHERE type = 'table'")
        return res.fetchall()

    def thesaurus(self, word: str) -> list[list[str]]:
        SQL_statement = f"""
            WITH search_synset AS (
                SELECT synset, word.wordid, word.lang
                FROM sense 
                    LEFT JOIN word
                        ON sense.wordid = word.wordid
                WHERE lemma = '{word}'
            )
            SELECT DISTINCT sense.synset, lemma
            FROM search_synset
                LEFT JOIN sense
                    ON search_synset.synset = sense.synset
                LEFT JOIN word
                    ON sense.wordid = word.wordid
                        AND search_synset.lang = word.lang
        """
        df = self.read_sql(SQL_statement)
        if len(df) == 0:
            raise KeyError(f"{word} not found!!")
        
        df = df[(df["lemma"] != word) & (df["lemma"].notna())]
        if len(df) == 0:
            raise KeyError(f"thesaurus of {word} not found!!")

        # there may be severals rows for the same word
        ret = []
        for (v, d) in df.groupby("synset"):
            ret.append(d["lemma"].tolist())
        
        return ret


In [77]:
class NINJAL:
    def __init__(self):
        colnames = [
            "id", # レコードID番号
            "headline_id", # 見出し番号
            "record_kind", #レコード種別,
            "kind", # 類,
            "division", # 部門,
            "item", # 中項目,
            "class", # 分類項目,
            "class_id", # 分類番号,
            "paragraph_id", # 段落番号,
            "small_paragraph_id", # 小段落番号,
            "word_id", # 語番号,
            "headline", # 見出し,
            "headline_body", # 見出し本体,
            "read", # 読み,
            "read_rev", # 逆読み,
        ]
        self.df = pd.read_csv("bunruidb.txt", encoding="CP932", names=colnames, header=None)

    def thesaurus(self, word: str) -> list[list[str]]:
        keys = ["record_kind", "kind", "division", "item", "class_id", "paragraph_id", "small_paragraph_id"]
        df_word = self.df[self.df["headline"] == word][keys]
        if len(df_word) == 0:
            raise KeyError(f"{word} not found!!")
        
        df_thesaurus = df_word.merge(self.df, how="left", on=keys)
        df_thesaurus = df_thesaurus[df_thesaurus["headline"] != word]
        if len(df_thesaurus) == 0:
            raise KeyError(f"thesaurus of {word} not found!!")
        
        # there may be severals rows for the same word
        ret = []
        for (v, d) in df_thesaurus.groupby(keys):
            ret.append(d["headline"].tolist())
        
        return ret

In [126]:
wn = WordNet()
wn.thesaurus("動物")

[['四つ脚',
  '毛の荒物',
  '獣畜',
  '4つ脚',
  'アニマル',
  '4つ足',
  '生物',
  '生き物',
  '獣',
  '珍獣',
  '生類',
  '生体',
  '四つ足',
  '鳥獣']]

In [78]:
ninja = NINJAL()
ninja.thesaurus("動物")

[['動物', 'アニマル', '鳥獣', '禽獣', '魚介'],
 ['畜類', '畜生', '動物', '獣畜'],
 ['植物体', '動物', '植物']]

## 2.2 
> Nグラム言語モデルで単語列が生起する確率を求めるとき, コーパス中に現れない単語が出現する問題をゼロ頻度問題と呼ぶ. このようなときにどのような対象法が考えられているのだろうか.

- [smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)

## 2.3
> 近年ではNグラム言語モデル以外にも様々な言語モデルが提案されている. どのような言語モデルが他に存在するのか, 調べてみよう.

 https://en.wikipedia.org/wiki/Language_model